# Real Experiment of CIFAR10 Classification with a MLP

File to perform experiments

## Imports

In [ ]:
import os
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm

from models.GaussianProcess import GaussianProcess
from acquisition_functions.MES import mes_acq, basic_mes_acq
from acquisition_functions.PESMO import pesmo_acq
from acquisition_functions.MESMO import mesmo_acq
from arguments.arguments import MainArguments

from MOObenchmark import MOOackley, MOOexponential_ackley, MOOexponential
from MOO_MLPfunction import evalMLPCIFAR2
from utils.calc_pareto import get_pareto_undominated_by, getSetfromFront

from models.MOOEvaluationProblem import MOOEvaluationProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
from pymoo.optimize import minimize

## Algorithm Arguments

In [ ]:
d = 2
    
seed = 0
np.random.seed(seed)

total_iter = 2
initial_iter = 1

lower_bound = -6
upper_bound = 0

lowerBounds = [lower_bound]*d
upperBounds = [upper_bound]*d


## Evaluation

In [ ]:
def random_acq(GP: GaussianProcess, **kwargs):
    while True:
        x_rand = np.random.uniform(GP.lowerBounds, GP.upperBounds, GP.d)
        if GP.X is None or not x_rand in GP.X:
            break
    return x_rand, 0

## N experiments

In [ ]:
df = None
n_experiments = 1
folder = "./MLPCIFAR2/"
savename = "basic_mes"

i = 0

def evaluation(x):
    return evalMLPCIFAR2(x)

O = 2
C = 0

while i<n_experiments:
    print(i, time.ctime())
    

   
    # try:
    ### GPs Initialization
    GP = GaussianProcess(O, C, d, lowerBounds, upperBounds, noise_variance=2e-6)

    #### Initial samples, at least 1
    for l in range(initial_iter):
        ## Get random evaluation point
        while True:
            x_rand = np.random.uniform(lowerBounds[0], upperBounds[0], d)
            if GP.X is None or not x_rand in GP.X:
                break
        ## EVALUATION OF THE OUTSIDE FUNCTION
        y_rand = evaluation(x_rand)
        GP.addSample(x_rand,y_rand)

    GP.updateGP()
    GP.optimizeKernel()

    d_ek, dm_ek = GP.evaluateNoRealPareto(showparetos=True)
    row = {
        'exp_id'   : i,
        'savename' : savename,
        'ns'       : len(GP.X),
        'x'        : x_rand,
        'y'        : y_rand,
        'time'     : float("NaN"),
        'd_ek'     : d_ek,
        'dm_ek'    : dm_ek
    }
    df_temp = pd.DataFrame({k: [v] for k, v in row.items()})

    for l in range(total_iter):

        ## Search of the best acquisition function
        start = time.time()
        x_best, acq_best = basic_mes_acq(GP)
        end = time.time()

        ## EVALUATION OF THE OUTSIDE FUNCTION
        y_best = evaluation(x_best)

        ## UPDATE
        GP.addSample(x_best,y_best)     ## Add new sample to the model
        GP.updateGP()                  ## Update data on the GP regressor
        GP.optimizeKernel()             ## Optimize kernel hyperparameters


        ## Evaluate Pareto (distances and hypervolumes)
        d_ek, dm_ek = GP.evaluateNoRealPareto(showparetos=True)
        row = {
            'exp_id'   : i,
            'savename' : savename,
            'ns'       : len(GP.X),
            'x'        : x_best,
            'y'        : y_best,
            'time'     : end-start,
            'd_ek'     : d_ek,
            'dm_ek'    : dm_ek
        }

        df_temp = pd.concat([df_temp, pd.DataFrame({k: [v] for k, v in row.items()})])
    if df is None:
        df = df_temp
    else:
        df = pd.concat([df, df_temp])
    i+=1
    # except:
    #     print(i, len(GP.X), GP.X, GP.Y)
    #     print("NON INVERTIBLE MAtRIX")
df.to_csv(folder+savename+".csv")

In [ ]:
df